In [41]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import re
from pandas import DataFrame, Series
file_name = '1_all_2.csv'
homes_df = pd.read_csv(file_name)
homes_df.pop('Unnamed: 0')
# homes_df.pop('city')

0              1
1              2
2              3
3              4
4              5
5              6
6              7
7              8
8              9
9             10
10            11
11            12
12            13
13            14
14            15
15            16
16            17
17            18
18            19
19            20
20            21
21            22
22            23
23            24
24            25
25            26
26            27
27            28
28            29
29            30
           ...  
224273    226299
224274    226300
224275    226301
224276    226302
224277    226303
224278    226304
224279    226305
224280    226306
224281    226307
224282    226308
224283    226309
224284    226310
224285    226311
224286    226312
224287    226313
224288    226314
224289    226315
224290    226316
224291    226317
224292    226318
224293    226319
224294    226320
224295    226321
224296    226322
224297    226323
224298    226324
224299    226325
224300    2263

In [42]:
homes_df.index = np.arange(len(homes_df)) + 1
# homes_df.pop('unit_count')
# homes = homes_df.drop_duplicates(['price', 'sold_date', 'zipcode'])

## 数据范围： 2015.8.1 ~ 2017.9.1

In [43]:
homes_df.shape
bar = datetime(2017, 9, 1)
homes_df = homes_df[homes_df.sold_date.map(lambda x: datetime.strptime(x, '%Y-%m-%d')) < bar]
homes_df.index = np.arange(len(homes_df)) +1

In [44]:
bar = datetime(2015, 7, 31)
homes_df = homes_df[homes_df.sold_date.map(lambda x: datetime.strptime(x, '%Y-%m-%d')) > bar]

In [45]:
# change date 
start_time = datetime(2015, 8, 1)
homes_df.sold_date = homes_df.sold_date.map(lambda x: (datetime.strptime(x, '%Y-%m-%d') - start_time).days)

## 价格范围：>10万

In [46]:
homes_df = homes_df[homes_df.price > 100000]

In [48]:
homes_df.shape

(166003, 166)

## 去除5个STD以外的数据

In [49]:
todeletecolumns = ['unit_count', 'price', 'size', 'Beds', 'lot', 'garage_num', 'stories','room_count', 'lotDepth', 'lotWidth', 'hoa']
'''
homes_new = homes_df.copy()
summ = 0
for col in todeletecolumns:
    temp = homes_new[col].dropna()
    summ += len(temp[np.abs(temp - temp.mean()) > 5 * temp.std()])
print summ
'''
for col in todeletecolumns:
    temp = homes_df[col].dropna()
    homes_df = homes_df.drop(temp[np.abs(temp - temp.mean()) > 5 * temp.std()].index)

In [50]:
homes_df.shape

(163745, 166)

## 选择某些地区的数据

In [58]:
city = 'Los Angeles, Long Beach, Anaheim, Santa Ana, Irvine, Glendale, Torrance, Pasadena, Orange, Costa Mesa, Burbank, Carson, Santa Monica, Newport Beach, Tustin, Monterey Park, Gardena, Arcadia, Fountain Valley'
c = city.split(', ')
homes_selected = homes_df[homes_df.city.isin(c)]
homes_selected.shape

(45152, 166)

## write

In [59]:
homes_selected.head()

,sold_date,price,detail_url,city,zipcode,"Type and Style_[""Apartment""]","Type and Style_[""Condo""]","Type and Style_[""Cooperative""]","Type and Style_[""Miscellaneous""]","Type and Style_[""Mobile / Manufactured""]",...,"yard_""Garden""","yard_""Fenced Yard""","yard_""Lawn""",bath_full,bath_three-quarter,bath_half,high_school,"material_""Double Pane/Storm Windows""","material_""Skylight""",middle_school
35329,0,1568000.0,https://www.zillow.com/homedetails/1739-Elevad...,Arcadia,91006,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35334,0,300000.0,https://www.zillow.com/homedetails/427-Orange-...,Irvine,92618,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35336,0,700000.0,https://www.zillow.com/homedetails/59-Juneberr...,Irvine,92606,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35340,1,2775000.0,https://www.zillow.com/homedetails/560-S-Silve...,Anaheim,92807,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35341,1,650000.0,https://www.zillow.com/homedetails/2563-Calle-...,Tustin,92782,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:

homes_selected.to_csv('1_selected_1.csv')

## 选择重要的列

In [61]:
"""
1) sold_date                      0.477267
 2) zipcode                        0.204374
 3) num                            0.037938
 4) Type and Style_["Apartment"]   0.034041
 5) Type and Style_["Condo"]       0.019519
 6) Type and Style_["Cooperative"] 0.017182
 7) Type and Style_["Miscellaneous"] 0.015473
 8) Type and Style_["Mobile / Manufactured"] 0.015113
 9) Type and Style_["Multi Family"] 0.013602
10) Type and Style_["Multiple Occupancy"] 0.012124
11) Type and Style_["Other"]       0.011731
12) Type and Style_["Single Family"] 0.010860
13) Type and Style_["Townhouse"]   0.009640
14) size                           0.009491
15) Beds                           0.006890
16) built_date                     0.006682
17) unit_count                     0.006622
18) lot                            0.005707
19) garage_num                     0.005373
20) stories                        0.004376
21) heat_zoned                     0.003877
22) heat_wall                      0.003675
23) heat_stove                     0.002991
24) heat_baseborad                 0.002942
25) heat_floor                     0.002695
26) heat_gravity                   0.002072
"""
homes_selected_c = homes_selected[['sold_date','detail_url', 'price', 'zipcode', 'Type and Style_["Apartment"]', 'Type and Style_["Condo"]', 'Type and Style_["Cooperative"]', 
                         'Type and Style_["Miscellaneous"]', 'Type and Style_["Mobile / Manufactured"]',
                        'Type and Style_["Multi Family"]', 'Type and Style_["Multiple Occupancy"]',
                        'Type and Style_["Other"]', 'Type and Style_["Single Family"]', 'Type and Style_["Townhouse"]', 
                        'size', 'Beds', 'built_date', 'unit_count', 'lot', 'garage_num', 'stories', 'heat_zoned', 
                        'heat_wall', 'heat_stove', 'heat_baseborad', 'heat_floor', 'heat_gravity']]


In [63]:
homes_selected_c.shape

(45152, 27)

In [64]:
homes_selected_c.to_csv('1_selected_2.csv')

## 填缺失值

In [65]:
homes_selected_c.head()

,sold_date,detail_url,price,zipcode,"Type and Style_[""Apartment""]","Type and Style_[""Condo""]","Type and Style_[""Cooperative""]","Type and Style_[""Miscellaneous""]","Type and Style_[""Mobile / Manufactured""]","Type and Style_[""Multi Family""]",...,unit_count,lot,garage_num,stories,heat_zoned,heat_wall,heat_stove,heat_baseborad,heat_floor,heat_gravity
35329,0,https://www.zillow.com/homedetails/1739-Elevad...,1568000.0,91006,0,0,0,0,0,0,...,1.0,1350360.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
35334,0,https://www.zillow.com/homedetails/427-Orange-...,300000.0,92618,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
35336,0,https://www.zillow.com/homedetails/59-Juneberr...,700000.0,92606,0,0,0,0,0,0,...,0.0,2125.0,2.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
35340,1,https://www.zillow.com/homedetails/560-S-Silve...,2775000.0,92807,0,0,0,0,0,0,...,0.0,2265120.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
35341,1,https://www.zillow.com/homedetails/2563-Calle-...,650000.0,92782,0,0,0,0,0,0,...,0.0,2221.0,2.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
homes_selected_c.pop('detail_url')
homes_fill = homes_selected_c.fillna(homes_selected_c.mean())

## normalization

In [67]:
homes_5 = homes_fill.copy()
price = homes_5.pop('price')
logerror_df = np.log(price)
diff = logerror_df.max() - logerror_df.min()
logerror_df = (logerror_df - logerror_df.min()) * 1.0 / diff
transactiondate_df = homes_5.sold_date.copy()
homes_5 = (homes_5 - homes_5.min()) * 1.0/ (homes_5.max() - homes_5.min())
import pickle
data = {'logerror_df': logerror_df, 'train_df': homes_5, 'transactiondate_df': transactiondate_df}
with open('zillow-model-data-original-10', 'wb') as f:
    pickle.dump(data, f)

In [68]:
diff

3.7562125741825234

In [69]:
homes_5.isna().any().any()

False

In [70]:
homes_5.shape

(45152, 25)